# 크롤러

In [75]:
import pandas as pd
import requests

def get_bok_df(info, classification,start_data, end_data, specific_info):
    apikey = "HF0ML1Y9A0RN7R7K4JE2"
    url = 'https://ecos.bok.or.kr/api/StatisticSearch/' + apikey + '/json/kr/1/100/817Y002/D/20180101/20221231/010200000'
    response = requests.get(url)

    result = response.json()
    list_total_count=(int)(result['StatisticSearch']['list_total_count'])
    list_count=(int)(list_total_count/100) + 1
    rows=[]
    for i in range(0,list_count):
        start = str(i * 100 + 1)
        end = str((i + 1) * 100)
        url = 'https://ecos.bok.or.kr/api/StatisticSearch/' + apikey + '/json/kr/' \
                + start + '/' + end + f"/{info}/{classification}/{start_data}/{end_data}/{specific_info}" 
        response = requests.get(url)
        result = response.json()
        rows = rows + result['StatisticSearch']['row']
    df=pd.DataFrame(rows)
    return df


## 파라미터 정보

In [76]:
# 원화의 대미 달러 종가기준 연평균
param_uskor_exchange = ['731Y006', 'A', 2018, 2022, '0000003']

# M2 경제주체별 연평균, 단위 십억원
param_m2 = ['101Y015', 'A', 2018, 2022, 'BBHA00']

# 생산자물가지수 연평균, 2015년 PPI를 100으로 함.
param_PPI = ['404Y014', 'A', 2018, 2022, '*AA']


# 한국 정책금리 월단위
param_kr_policyratio = ['902Y006', 'M', 201801, 202212, 'KR']
# 미국 정책금리 월단위
param_us_policyratio = ['902Y006', 'M', 201801, 202212, 'US']

# # 한국은행 기준금리 일단위
# param_kor_standard_yield = ['722Y001', 'D', 20180101, 20221231, '0101000']
# # 국고채 3년물 금리 일단위 -> 집계필요
# param_kor_govrn_3ybond = ['817Y002', 'D', 20180101, 20221231, '010200000'] 

In [77]:
# 환율, 통화량, 생산자 물가지수 연평균
uskor_exchange_year = get_bok_df(*param_uskor_exchange)
m2_year = get_bok_df(*param_m2)
PPI_year = get_bok_df(*param_PPI)

# 월단위 정책금리
kr_policyratio_month  = get_bok_df(*param_kr_policyratio)
us_policyratio_month = get_bok_df(*param_us_policyratio)

### 최저임금 데이터 로드 - csv file

In [78]:
minimum_wage_1423 = pd.read_csv("C:/Users/USER_20211027/Desktop/final_project/github/Final-Project/data/고용노동부_연도별 최저임금_2014_2023.csv", encoding="utf-8")

## 데이터 프레임으로 저장

# DB 전송

In [79]:
def pythondf_to_DB(dataframe_name, database_name, desired_table_name):
	import pymysql
	from sqlalchemy import create_engine
	from sqlalchemy.orm import sessionmaker
	username = 'multi'
	password = 'Campus123!'
	hostname =  'ec2-15-152-211-160.ap-northeast-3.compute.amazonaws.com'
	database_name = database_name
	desired_table_name = desired_table_name
	cnx = pymysql.connect(user=username, password=password, host=hostname)
	cursor = cnx.cursor()
	engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
						.format(user=username,
								pw=password,
								db=database_name,
								host=hostname))
	Session = sessionmaker(bind=engine)
	session = Session()
	try:
		# Change df name
		dataframe_name.to_sql(desired_table_name, con=engine, if_exists='replace', index=False, chunksize = 1000)
		session.commit()
	except:
		session.rollback()
		raise
	finally:
		session.close()
	cursor.close()

## Data Lake upload

In [80]:
# def pythondf_to_DB(dataframe_name, "database_name", "desired_table_name")

# 연평균 원/달러 환율
pythondf_to_DB(uskor_exchange_year, "Data_Lake", "uskor_exchange_year")

# 연평균 통화량
pythondf_to_DB(m2_year, "Data_Lake", "m2_year")

# 연평균 생산자 물가
pythondf_to_DB(PPI_year, "Data_Lake", "PPI_year")

# 한국, 미국 월단위 정책금리
pythondf_to_DB(kr_policyratio_month, "Data_Lake", "kr_policyratio_month")
pythondf_to_DB(us_policyratio_month, "Data_Lake", "us_policyratio_month")

# 연별 최저임금
pythondf_to_DB(minimum_wage_1423, "Data_Lake", "minimum_wage_1423")

# # Data Warehouse - 테이블 변경

정책금리차 테이블 변경

In [49]:
kr_policyratio_month['TIME'] = pd.to_datetime(kr_policyratio_month['TIME'], format='%Y%m')
kr_policyratio_month['YEAR'] = kr_policyratio_month['TIME'].dt.year
kr_policyratio_month['DATA_VALUE'] = kr_policyratio_month['DATA_VALUE'].astype(float)
kr_policyratio_year = kr_policyratio_month.groupby('YEAR')['DATA_VALUE'].mean().round(4)

us_policyratio_month['TIME'] = pd.to_datetime(us_policyratio_month['TIME'], format='%Y%m')
us_policyratio_month['YEAR'] = us_policyratio_month['TIME'].dt.year
us_policyratio_month['DATA_VALUE'] = us_policyratio_month['DATA_VALUE'].astype(float)
us_policyratio_year = us_policyratio_month.groupby('YEAR')['DATA_VALUE'].mean().round(4)

In [50]:
# 한국정책금리 - 미국정책금리 -> 양수값이 나오면 한국정책금리 > 미국정책금리
kr_minus_us_policyratio = kr_policyratio_year - us_policyratio_year
krus_diff_policyratio = pd.DataFrame({'year': kr_minus_us_policyratio.index, 'kr_minus_us_rate': kr_minus_us_policyratio.values})

# # 결과 출력
# print(df_kr_minus_us_policyratio)

달러원 환율 테이블 변경

In [21]:
uskor_exchange_year_average = uskor_exchange_year[uskor_exchange_year.ITEM_NAME2=="평균자료"]

In [22]:
exchangerate_USDKRX_avg = uskor_exchange_year_average[['TIME', 'DATA_VALUE']]
exchangerate_USDKRX_avg = exchangerate_USDKRX_avg.rename(columns={'DATA_VALUE': 'uskor_exchange_average', 'TIME': 'year'})

M2 통화량

In [25]:
m2_year_df = m2_year[['TIME', 'DATA_VALUE']]
m2_year_df = m2_year_df.rename(columns={'DATA_VALUE': 'm2_year', 'TIME': 'year'})


생산자 물가지수

In [31]:
PPI_year_df = PPI_year[['TIME', 'DATA_VALUE']]
PPI_year_df = PPI_year_df.rename(columns={'DATA_VALUE': 'PPI_year', 'TIME': 'year'})

최저임금 테이블 변경

In [35]:
minimum_wage_1423

,����,����.1,�ð���
0,1,2023,9620
1,2,2022,9160
2,3,2021,8720
3,4,2020,8590
4,5,2019,8350
5,6,2018,7530
6,7,2017,6470
7,8,2016,6030
8,9,2015,5580
9,10,2014,5210


In [36]:
minimum_wage_1423_df = minimum_wage_1423[['����.1', '�ð���']]
minimum_wage_1423_df = minimum_wage_1423_df.rename(columns={'����.1': 'year', '�ð���': 'minimum_wag'})

### 변경완료된 df - 통화량, 환율, 최저임금, 정책금리차

In [55]:
minimum_wage_1423 = minimum_wage_1423_df[1:6]

In [64]:
list = [minimum_wage_1423,
exchangerate_USDKRX_avg,
PPI_year_df,
krus_diff_policyratio]

for i in list:
    i['year'] = i['year'].astype(int)

C:\Users\USER_20211027\AppData\Local\Temp\ipykernel_2184\1340938773.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i['year'] = i['year'].astype(int)


In [65]:
merged_df = pd.merge(minimum_wage_1423, exchangerate_USDKRX_avg, on='year')
merged_df = pd.merge(merged_df, PPI_year_df, on='year')
merged_df = pd.merge(merged_df, krus_diff_policyratio, on='year')

# 인덱스 재설정
merged_df = merged_df.reset_index(drop=True)

# 결과 확인
print(merged_df)

   year  minimum_wag uskor_exchange_average PPI_year  kr_minus_us_rate
0  2022         9160                 1292.2   118.78            0.2292
1  2021         8720                1144.61    109.6            0.5208
2  2020         8590                1180.01   103.03            0.2917
3  2019         8350                1166.11    103.5           -0.5625
4  2018         7530                1100.58   103.48           -0.2916


In [81]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    5 non-null      object 
 1   minimum_wag             5 non-null      int64  
 2   uskor_exchange_average  5 non-null      float64
 3   PPI_year                5 non-null      float64
 4   kr_minus_us_rate        5 non-null      float64
dtypes: float64(3), int64(1), object(1)
memory usage: 328.0+ bytes


In [67]:
merged_df['uskor_exchange_average'] = merged_df['uskor_exchange_average'].astype(float)
merged_df['PPI_year'] = merged_df['PPI_year'].astype(float)

In [71]:
merged_df['year'] = merged_df['year'].astype(str)

## Data Warehouse - upload

In [86]:
# def pythondf_to_DB(dataframe_name, "database_name", "desired_table_name")

# 연평균 원/달러 환율
pythondf_to_DB(merged_df1, "Data_Lake", "economic_indicators")

In [82]:
merged_df

,year,minimum_wag,uskor_exchange_average,PPI_year,kr_minus_us_rate
0,2022,9160,1292.20,118.78,0.2292
1,2021,8720,1144.61,109.60,0.5208
2,2020,8590,1180.01,103.03,0.2917
3,2019,8350,1166.11,103.50,-0.5625
4,2018,7530,1100.58,103.48,-0.2916


In [84]:
merged_df1 = merged_df.rename(columns={'kr_minus_us_rate': 'kor_usa_ir_diff'})
merged_df1['kor_usa_ir_diff'] = merged_df1['kor_usa_ir_diff'].round(decimals=4)

In [85]:
merged_df1

,year,minimum_wag,uskor_exchange_average,PPI_year,kor_usa_ir_diff
0,2022,9160,1292.20,118.78,0.2292
1,2021,8720,1144.61,109.60,0.5208
2,2020,8590,1180.01,103.03,0.2917
3,2019,8350,1166.11,103.50,-0.5625
4,2018,7530,1100.58,103.48,-0.2916
